In [1]:
#########################################################################
# define filename and use gmt grdinfo to display information about grid #
#########################################################################

FILENAME = 'threesis_19950924_20010902.grd'
# ! indicates a shell command, -La scans the date to report real z stats
! gmt grdinfo $FILENAME -La

grdinfo [WARNING]: 2971229 Multiple modes found in the grid
threesis_19950924_20010902.grd: Title: ERS2 Rangechange Interferogram stack, 19950924-20010902
threesis_19950924_20010902.grd: Command: xyz2grd -V -R-122.60/-120.4008333333/42.9/45.09916667 -I3c/3c tmp.xyz -Grng_avg_9.grd -Ddd/dd/mm/1/=/Demeaned Range Change
threesis_19950924_20010902.grd: Remark: WGS84, 20.5 deg incidence angle, 192.7 deg Az satellite heading, nodata=-9999
threesis_19950924_20010902.grd: Gridline node registration used [Cartesian grid]
threesis_19950924_20010902.grd: Grid file format: nf = GMT netCDF format (32-bit float), CF-1.7
threesis_19950924_20010902.grd: x_min: -122.600000002 x_max: -120.400833335 x_inc: 0.000833333333346 name: deg n_columns: 2640
threesis_19950924_20010902.grd: y_min: 42.9 y_max: 45.09916667 y_inc: 0.000833333334596 name: deg n_rows: 2640
threesis_19950924_20010902.grd: z_min: -9999 z_max: 29.9956474304 name: mm
threesis_19950924_20010902.grd: scale_factor: 1 add_offset: 0
threesis_19

In [31]:
#######################################
# make a color plot to visualize data #
#######################################

# use automatic=True to find the bounds of the region from the grid file,
# or use automatic=False to supply different bounds to plot 
automatic = False

if automatic:
    # first store the region that the grid file encompasses as variables to pass to GMT
    XMIN = ! gmt grdinfo $FILENAME -Cn -o0
    XMAX = ! gmt grdinfo $FILENAME -Cn -o1
    YMIN = ! gmt grdinfo $FILENAME -Cn -o2
    YMAX = ! gmt grdinfo $FILENAME -Cn -o3
    # the above gmt grdinfo command stores the values in a list. we only want the string
    XMIN = XMIN[0]
    XMAX = XMAX[0]
    YMIN = YMIN[0]
    YMAX = YMAX[0]
else:
    # define bounds to plot
    XMIN = -122.1
    XMAX = -121.4
    YMIN = 43.7
    YMAX = 44.5

# feed the data to gmt, write to color_grid.png
! gmt begin color_grid png

    # create the map frame and set the region and projection
!    gmt basemap -R$XMIN/$XMAX/$YMIN/$YMAX -JM15c -Baf

    # plot topography using gmt built-in dataset at the highest resolution
!    gmt grdimage @earth_relief_01s -Cgray -I+d

    # make a new colormap that only covers the values of interest for the grid
!    gmt makecpt -Cseis -T-140/35

    # plot grid as colored map using grdimage, transparency 60%
!    gmt grdimage $FILENAME -t60 -Q

    # show the colorbar, ticks every 25 mm, add title
!    gmt colorbar -B25 -Bx+l"range change [mm]"

#     # to plot a fake point source 
# !    echo -121.835 44.090 > t.dat
# !    gmt plot t.dat -Gblack -Sc0.5c

! gmt end show

grdblend [WARNING]: File @N40W130.earth_relief_15s_p.nc has different increments (0.00416666666667/0.00416666666667) than the output grid (0.000277777777778/0.000277777777778) - must resample
grdblend [WARNING]: File @N40W130.earth_relief_15s_p.nc coordinates are phase-shifted w.r.t. the output grid - must resample


In [21]:
########################################
# subsample the grid file to 80 points #
########################################
"""
# for a simply regridding use gmt to resample the grid as below:
# define output file name
OUTFILE = 'resampled.grd'
# resample using grdsample, -I denotes number of nodes, -R denotes subregion, 
! gmt grdsample $FILENAME -I0.02 -R$XMIN/$XMAX/$YMIN/$YMAX -G$OUTFILE
# inspect the details of the new file
! gmt grdinfo $OUTFILE -La
"""

# define region of interest to subsample
XMIN = -122.1
XMAX = -121.4
YMIN = 43.7
YMAX = 44.5

# write grd file to text to sample the grid pythonically 
! gmt grd2xyz $FILENAME -R$XMIN/$XMAX/$YMIN/$YMAX > grid.xyz

# load the xyz file
# to np or dask array?

# for this case I want to sample the grid more densely near the deformation source
deformation_Source = (-121.835, 44.090) # this is a guess

# find all points within __ distance to the deformation source
#  np.where or dask where

# randomly sample those points 40 times

# randomly sample remaining points > ___ distance from the deformation source 40 times

# save as xyz file


"\n# for a simply regridding use gmt to resample the grid as below:\n# define output file name\nOUTFILE = 'resampled.grd'\n# resample using grdsample, -I denotes number of nodes, -R denotes subregion, \n! gmt grdsample $FILENAME -I0.02 -R$XMIN/$XMAX/$YMIN/$YMAX -G$OUTFILE\n# inspect the details of the new file\n! gmt grdinfo $OUTFILE -La\n"

In [20]:
# plot xyz file in gmt plot & grdimage

grdblend [WARNING]: File @N40W130.earth_relief_15s_p.nc has different increments (0.00416666666667/0.00416666666667) than the output grid (0.000277777777778/0.000277777777778) - must resample
grdblend [WARNING]: File @N40W130.earth_relief_15s_p.nc coordinates are phase-shifted w.r.t. the output grid - must resample


In [ ]:
#################################
# plot the subsampled grid file #
#################################

# feed the data to gmt, write to resampled_grid.png
! gmt begin resampled_grid png

    # create the map frame and set the region and projection
!    gmt basemap -R$XMIN/$XMAX/$YMIN/$YMAX -JM15c -Baf

    # plot topography using gmt built-in dataset at the highest resolution
!    gmt grdimage @earth_relief_01s -Cgray -I+d

    # make a new colormap that only covers the values of interest for the grid
!    gmt makecpt -Cseis -T-140/35

    # plot grid as colored map using grdimage, transparency 60%
!    gmt grdimage resampled.grd -t60

    # show the colorbar, ticks every 25 mm, add title
!    gmt colorbar -B25 -Bx+l"range change [mm]"

! gmt end show